In [ ]:
# PROCESOS
# 1. PLACE_NAME
    # 1.1 'place_with_parent_names' 
    # 1.2 calidad_dato
# 2. GEONAMES_ID
    # 2.1 validar que No se asigne un mismo geonames_id a distintos valores de place_name_new
    # 2.2 Normalizacion de valores nulos
# 3. LAT-LON
# 4. TIPO_DE_CAMBIO
# 5. PRECIO_M2

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
# importo archivo
data_location = "C:/Users/Fernando/Documents/DIGITAL_HOUSE/CONTENIDO/DESAFIO_I/properati.csv"
data = pd.read_csv(data_location, sep=",", encoding="utf-8")

In [3]:
# abro el archivo
data.head(5)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...
4,4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,...,35.0,1828.571429,1828.571429,NaN,NaN,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...


In [4]:
# tipo de datos de cada columna
data.dtypes

Unnamed: 0                      int64
operation                      object
property_type                  object
place_name                     object
place_with_parent_names        object
country_name                   object
state_name                     object
geonames_id                   float64
lat-lon                        object
lat                           float64
lon                           float64
price                         float64
currency                       object
price_aprox_local_currency    float64
price_aprox_usd               float64
surface_total_in_m2           float64
surface_covered_in_m2         float64
price_usd_per_m2              float64
price_per_m2                  float64
floor                         float64
rooms                         float64
expenses                      float64
properati_url                  object
description                    object
title                          object
image_thumbnail                object
dtype: objec

In [5]:
# renombro la columna que no tiene nombre
data = data.rename({           
    'Unnamed: 0': 'Id_caso'
}, axis=1)

In [6]:
# % de nulos por columna
print ('Porcentaje de nulos por variable')
round(data.isnull().sum() / data.shape[0]*100,1)

Porcentaje de nulos por variable


Id_caso                        0.0
operation                      0.0
property_type                  0.0
place_name                     0.0
place_with_parent_names        0.0
country_name                   0.0
state_name                     0.0
geonames_id                   15.4
lat-lon                       42.5
lat                           42.5
lon                           42.5
price                         16.8
currency                      16.8
price_aprox_local_currency    16.8
price_aprox_usd               16.8
surface_total_in_m2           32.4
surface_covered_in_m2         16.4
price_usd_per_m2              43.4
price_per_m2                  27.7
floor                         93.5
rooms                         60.9
expenses                      88.2
properati_url                  0.0
description                    0.0
title                          0.0
image_thumbnail                2.6
dtype: float64

In [7]:
# descriptivo sobre algunas variables
medidas = data[["price_aprox_usd","surface_total_in_m2","surface_covered_in_m2", "price_usd_per_m2","price_per_m2"]].describe()
print(medidas)

       price_aprox_usd  surface_total_in_m2  surface_covered_in_m2  \
count     1.008100e+05         81892.000000          101313.000000   
mean      2.397006e+05           233.795328             133.050181   
std       3.913239e+05          1782.222147             724.351479   
min       0.000000e+00             0.000000               0.000000   
25%       8.973388e+04            50.000000              45.000000   
50%       1.450000e+05            84.000000              75.000000   
75%       2.650000e+05           200.000000             150.000000   
max       4.654544e+07        200000.000000          187000.000000   

       price_usd_per_m2  price_per_m2  
count      68617.000000  8.765800e+04  
mean        2160.086916  6.912216e+03  
std         2759.288621  2.837864e+04  
min            0.600000  1.510204e+00  
25%         1218.181818  1.550000e+03  
50%         1800.000000  2.213115e+03  
75%         2486.411765  3.355549e+03  
max       206333.333333  4.000000e+06  


In [8]:
# valores 
medida_property_type = data.property_type.value_counts()
print(medida_property_type) 
# ...........terminar los otros

apartment    71065
house        40268
PH            5751
store         4136
Name: property_type, dtype: int64


In [9]:
# vamos a determinadar si la variable 'place_name' tiene un dato 'confiable' o 'completo', 
# para eso separamos la variable 'place_with_parent_names' en partes, por el simbolo | (pipe) 
# separo la variable place_with_parent_names, sumando campos nuevos y ya trabajando sobre le nuevo dataframe
data_trabajo = data.join(data["place_with_parent_names"].str.split('|', expand=True).rename(columns={
                                                                                                1:'Pais', 
                                                                                                2:'Zona', 
                                                                                                3:'Partido_barrio', 
                                                                                                4:'Localidad', 
                                                                                                5:'Obs_localidad', 
                                                                                                6:'Descarte'}))
data_trabajo.drop(['Descarte'], axis='columns', inplace=True)

In [10]:
# validamos que la variable place_name sea igual a el ultimo valor informado sobre place_with_parent_names
# ya separado por el proceso anterior
# lo recorro desde la ultima variable hasta encontrar un NO nulo y grabo el valor
# 1 - Obs_localidad
# 2 - Localidad
# 3 - Partido_barrio
# 4 - Zona
# creo una nueva variable que voy a ir completando a medida que valido que cada 
# variable tenga valor empezando desde la última
data_trabajo["place_name_new"] = 'validar'

#para cada variable nuevo valido null o ''
#---------------------------------------------------------------------------
# creo mascara de los casos donde la varible 'x' tenga null o ''
mask_data_Obs_localidad_null =  (data_trabajo.Obs_localidad.isnull()) | (data_trabajo.Obs_localidad == '')
# creo otra mascara con origen en la mascara enterior para quedarme con los 
# que tienen valor en la variable 'x'
mask_data_Obs_localidad_notnull = mask_data_Obs_localidad_null == False
print ('Actualización de place_name_new')
print ('--------------------------------')

print ('Obs_localidad')
print ('Cantidad de casos con Obs_localidad informada: ' + str(mask_data_Obs_localidad_notnull.sum()))
# creo una mascara sobre la variable que voy a ir completando con el valor obtenido
mask_data_place_name_new_validar = data_trabajo.place_name_new == 'validar'
print ('Cantidad de casos con place_name_new original :' + str(mask_data_place_name_new_validar.sum()))
# actualizo los valores sobre mi nueva variable con los que encontré en la variable 'x'
data_trabajo.loc[mask_data_Obs_localidad_notnull&mask_data_place_name_new_validar, "place_name_new"] = data_trabajo.Obs_localidad
# actualizo la mascara sobre mi nueva variable para contar y ver si todo resultó ok!
mask_data_place_name_new_validar = data_trabajo.place_name_new == 'validar'
print ('Cantidad de casos con place_name_new posterior para analizar: ' + str(mask_data_place_name_new_validar.sum()))
# la cantidad de casos sobre la variable nueva con valor 'validar' era de 121197
# La cantidad de casos sobre la misma variable con valor 'validar' despues de actualizar fue de 120649
# El delta corresponde a los valores con origen en el primer campo Obs_localidad ... 548

#---------------------------------------
print ('--------------------------------')
# 2 - Localidad
mask_data_Localidad_null =  (data_trabajo.Localidad.isnull()) | (data_trabajo.Localidad == '')
mask_data_Localidad_notnull = mask_data_Localidad_null == False
print ('Localidad')
print ('Cantidad de casos con Localidad informada: ' + str(mask_data_Localidad_notnull.sum()))
mask_data_place_name_new_validar = data_trabajo.place_name_new == 'validar'
print ('Cantidad de casos con place_name_new original: ' +  str(mask_data_place_name_new_validar.sum()))
data_trabajo.loc[mask_data_Localidad_notnull&mask_data_place_name_new_validar, "place_name_new"] = data_trabajo.Localidad
mask_data_place_name_new_validar = data_trabajo.place_name_new == 'validar'
print ('Cantidad de casos con place_name_new posterior para analizar: ' + str(mask_data_place_name_new_validar.sum()))
#---------------------------------------
print ('--------------------------------')
# 3 - Partido_barrio
mask_data_Partido_barrio_null =  (data_trabajo.Partido_barrio.isnull()) | (data_trabajo.Partido_barrio == '')
mask_data_Partido_barrio_notnull = mask_data_Partido_barrio_null == False
print ('Partido_barrio')
print ('Cantidad de casos con Partido_barrio informada: ' + str(mask_data_Partido_barrio_notnull.sum()))
mask_data_place_name_new_validar = data_trabajo.place_name_new == 'validar'
print ('Cantidad de casos con place_name_new original: ' + str(mask_data_place_name_new_validar.sum()))
data_trabajo.loc[mask_data_Partido_barrio_notnull&mask_data_place_name_new_validar, "place_name_new"] = data_trabajo.Partido_barrio
mask_data_place_name_new_validar = data_trabajo.place_name_new == 'validar'
print ('Cantidad de casos con place_name_new posterior para analizar: ' + str(mask_data_place_name_new_validar.sum()))
#---------------------------------------
print ('--------------------------------')
# 4 - Zona
mask_data_Zona_null =  (data_trabajo.Zona.isnull()) | (data_trabajo.Zona == '')
mask_data_Zona_notnull = mask_data_Zona_null == False
print ('Zona')
print ('Cantidad de casos con Zona informada: ' + str(mask_data_Zona_notnull.sum()))
mask_data_place_name_new_validar = data_trabajo.place_name_new == 'validar'
print ('Cantidad de casos con place_name_new original: ' + str(mask_data_place_name_new_validar.sum()))
data_trabajo.loc[mask_data_Zona_notnull&mask_data_place_name_new_validar, "place_name_new"] = data_trabajo.Zona
mask_data_place_name_new_validar = data_trabajo.place_name_new == 'validar'
print ('Cantidad de casos con place_name_new posterior para analizar: ' + str(mask_data_place_name_new_validar.sum()))
#----------------------------------------------------
print ('--------------------------------')
print ('Cantidad casos que coinciden entre place_name_new y place_name')
# valido si place_name_new da igual que place_name, si contamos lo mismo es que da igual
mask_place_name_validacion =  data_trabajo.place_name_new == data_trabajo.place_name
print(mask_place_name_validacion.value_counts())  
mask_place_name_validacion_falso = mask_place_name_validacion == False
print ('Vemos que casuiticas encontramos sobre los casos que no encontramos:')
print(data_trabajo.loc[mask_place_name_validacion_falso, ["Id_caso","place_with_parent_names","place_name", "place_name_new"]].\
     groupby(["place_with_parent_names"])["Id_caso"].count())
print('Vemos que son todos del mismo place_with_parent_names de Tigre')
# lo normalizamos
print ('Los normalizamos sobre la variable place_name_new')
data_trabajo.loc[mask_place_name_validacion_falso, "place_name_new"] = 'Tigre'
data_trabajo.loc[mask_place_name_validacion_falso, "Localidad"] = 'Tigre'
print ('Validamos la actualización:')
print(data_trabajo.loc[mask_place_name_validacion_falso, ["Id_caso", "place_name_new"]]\
     .groupby(["place_name_new"])["Id_caso"].count())

Actualización de place_name_new
--------------------------------
Obs_localidad
Cantidad de casos con Obs_localidad informada: 548
Cantidad de casos con place_name_new original :121220
Cantidad de casos con place_name_new posterior para analizar: 120672
--------------------------------
Localidad
Cantidad de casos con Localidad informada: 40417
Cantidad de casos con place_name_new original: 120672
Cantidad de casos con place_name_new posterior para analizar: 80803
--------------------------------
Partido_barrio
Cantidad de casos con Partido_barrio informada: 116440
Cantidad de casos con place_name_new original: 80803
Cantidad de casos con place_name_new posterior para analizar: 4780
--------------------------------
Zona
Cantidad de casos con Zona informada: 121220
Cantidad de casos con place_name_new original: 4780
Cantidad de casos con place_name_new posterior para analizar: 0
--------------------------------
Cantidad casos que coinciden entre place_name_new y place_name
True     121197

In [11]:
# nueva variable para validar la calidad del dato place_with_parent_names luego de separarlo 
data_trabajo["calidad_dato"] = False
# dependiendo de la zona que estamos analizando entendemos que la cantidad de info que necesitamos
# para ['Bs.As. G.B.A. Zona Norte','Bs.As. G.B.A. Zona Oeste','Bs.As. G.B.A. Zona Sur'] el mínimo tendria que ser Localidad
# para  ['Buenos Aires Costa Atlántica'] y en especial MDQ el mínimo tendria que ser Localidad
# para el resto_zonas Partido_barrio
# se va actualizando la variable creada (calidad_dato) a True que identifica los casos sobre los que necesitamos mayor informacion

# para ['Bs.As. G.B.A. Zona Norte','Bs.As. G.B.A. Zona Oeste','Bs.As. G.B.A. Zona Sur']
# zonas a traves de una lista de valores
valores_zona = ['Bs.As. G.B.A. Zona Norte','Bs.As. G.B.A. Zona Oeste','Bs.As. G.B.A. Zona Sur']
# mascara sobre la lista de valores y que localidad este nulo o ''
mask_Localidad = ([x in valores_zona for x in data_trabajo.Zona]) & ((data_trabajo.Localidad.isnull())|(data_trabajo.Localidad == ''))
# actualizo la variable creada sobre los True por la mascara
data_trabajo.loc[mask_Localidad, "calidad_dato"] = True
# ----------------------------------------------------------------------
# caso especial de MDQ
valores_zona_mdq = ['Buenos Aires Costa Atlántica']
mask_Localidad_mdq = ([x in valores_zona_mdq for x in data_trabajo.Zona])\
& (data_trabajo.Partido_barrio == 'Mar del Plata') & ((data_trabajo.Localidad.isnull()) | (data_trabajo.Localidad == ''))
data_trabajo.loc[mask_Localidad, "calidad_dato"] = True
# ----------------------------------------------------------------------
#resto_zonas
valores_zona_resto = ['Buenos Aires Costa Atlántica','Buenos Aires Interior','Capital Federal','Resto de provincias']
mask_Partido_barrio = ([x in valores_zona_resto for x in data_trabajo.Zona]) & ((data_trabajo.Partido_barrio.isnull())|\
                                                                                (data_trabajo.Partido_barrio == ''))
mask_calidad_dato_false = data_trabajo.calidad_dato == False
data_trabajo.loc[mask_Partido_barrio & mask_calidad_dato_false, "calidad_dato"] = True
# ----------------------------------------------------------------------
# todos los casos con True corresponden a los registros sobre los que nos falta informacion referida a 
# la geolocalización de la propiedad
print ('Cantidad de casos a los que le falta información de localización:')
print ('-----------------------------------------------------------------')
print(data_trabajo.calidad_dato.value_counts())
# mascara sobre casos que nos faltaria info
mask_faltante = (data_trabajo.calidad_dato == True)
data_faltante = data_trabajo.loc[mask_faltante]
#print (data_faltante)
# agrupo por Pais, Zona, Partido_barrio los casos que no tenemos otra info
groupby_df_faltante = data_faltante.groupby(['Pais',  'Zona', 'Partido_barrio'])["Id_caso"].count().\
sort_values(ascending=False)
print ('Agrupación de los casos a los que le falta información de localización:')
print ('-----------------------------------------------------------------------')
groupby_df_faltante


Cantidad de casos a los que le falta información de localización:
-----------------------------------------------------------------
False    107421
True      13799
Name: calidad_dato, dtype: int64
Agrupación de los casos a los que le falta información de localización:
-----------------------------------------------------------------------


Pais       Zona                          Partido_barrio     
Argentina  Bs.As. G.B.A. Zona Norte      Tigre                  2359
                                         Pilar                  1857
           Capital Federal                                      1297
           Bs.As. G.B.A. Zona Norte      Escobar                1037
           Bs.As. G.B.A. Zona Sur        La Plata                767
           Bs.As. G.B.A. Zona Norte      San Isidro              641
           Bs.As. G.B.A. Zona Oeste      Morón                   582
                                         Ituzaingó               474
           Bs.As. G.B.A. Zona Norte      San Fernando            437
           Bs.As. G.B.A. Zona Oeste      Moreno                  434
           Bs.As. G.B.A. Zona Norte      San Miguel              417
                                         General San Martín      382
           Bs.As. G.B.A. Zona Sur        Lomas de Zamora         276
           Bs.As. G.B.A. Zona Oeste      M

In [12]:
# variable geonames_id,  corresponde al id asignado a cada place_name
# primero vamos a validar que No se asigne un mismo geonames_id a distintos valores de place_name_new
mask_geonames_id_nonull = data_trabajo.geonames_id.notnull()
df_geonames_id_place_name_ag = data_trabajo.loc[mask_geonames_id_nonull]

grouped = df_geonames_id_place_name_ag.groupby(['geonames_id',  'place_name_new'])
grouped1 = grouped['Id_caso'].agg([np.size])

df_agrupado2 = grouped1.groupby(['geonames_id']).filter(lambda grp: grp["size"].count() > 1)
print ('Casos que un mismo geonames_id tiene asignado más de un place_name')
print ('------------------------------------------------------------------')
print (df_agrupado2)

# vemos que existen 5 valores de geonames_id que tiene mas de un valor de place_name
# por lo que nos vemos en la necesidad de analizar un poco mas, para ver si corresponden a la misma zona
# y ver que trabajo hacemos sobre ellos


Casos que un mismo geonames_id tiene asignado más de un place_name
------------------------------------------------------------------
                                          size
geonames_id place_name_new                    
3428927.0   San Jose                         5
            San José                        11
3430234.0   Palermo                       2885
            Palermo Soho                   394
3433359.0   Ituzaingó                      832
            Villa Udaondo                   25
3435548.0   Centro                         231
            Centro / Microcentro           223
3435907.0   Bs.As. G.B.A. Zona Norte       222
            Bs.As. G.B.A. Zona Oeste        65
            Bs.As. G.B.A. Zona Sur          24
            Buenos Aires Costa Atlántica    27
            Buenos Aires Interior          106


In [13]:
print ('Comenzamos con el proceso de análisis y actualización de los los casos:')
print ('-----------------------------------------------------------------------')
print ('geonames_id = 3428927')
print ('----------------------')
# geonames_id = 3428927
    # mascara de id
mask_geonames_id  = data_trabajo.geonames_id == 3428927
    # traemos la informacion de esos casos 
ver = data_trabajo.loc[mask_geonames_id,['place_with_parent_names']]
print (ver.place_with_parent_names.value_counts())
print ('Validamos que para esos place_with_parent_names no existan otros geonames_id')
    # Validamos que para esos place_with_parent_names no existan otros geonames_id
valores = ['|Argentina|Bs.As. G.B.A. Zona Sur|Lomas de Zamora|San José|','|Argentina|Bs.As. G.B.A. Zona Sur|\
Almirante Brown|San Jose|']
mask_place_with_parent_names  =  ([x in valores for x in data_trabajo.place_with_parent_names]) 
ver1 = data_trabajo.loc[mask_place_with_parent_names,['geonames_id'] ]
print (ver1.geonames_id.value_counts())
print ('Resultado ... como son de la misma zona y no existen mask_place_with_parent_names para otros geonames_id')
print ('Validamos en google que la localidad de "San Jose" corresponde al partido de "Almirante Brown"')
print ('Para esos registros mask_place_with_parent_names = |Argentina|Bs.As. G.B.A. Zona Sur|Lomas de Zamora|San José|')
print ('Actualizamos la variable Partido_barrio = "Almirante Brown"')
print ('Actualizamos la variable Localidad = "San Jose"')
print ('Actualizamos la variable place_name_new = "San Jose"')
    
    # resultado ... como son de la misma zona y no existen mask_place_with_parent_names para otros geonames_id
    # validamos en google que la localidad de 'San Jose' corresponde al partido de 'Almirante Brown'
    # para esos registros mask_place_with_parent_names = |Argentina|Bs.As. G.B.A. Zona Sur|Lomas de Zamora|San José|
    # actualizamos la variable Partido_barrio = 'Almirante Brown'
    # actualizamos la variable Localidad = 'San Jose'
    # actualizamos la variable place_name_new = 'San Jose'

# creamos una veriable igual a geonames_id sobre le df para trabajar sobre esa
data_trabajo['geonames_id_new'] = data_trabajo.geonames_id
# mascara de geonames_id 
mask_geonames_id  = data_trabajo.geonames_id == 3428927
mask_place_with_parent_names = data_trabajo.place_with_parent_names == '|Argentina|Bs.As. G.B.A. Zona Sur|\
Lomas de Zamora|San José|'
# actualizamos el valor de Partido_barrio para los casos de la mascara
print ('Cantidad de casos para actualizar:')
data_antes = data_trabajo.loc[mask_geonames_id & mask_place_with_parent_names,['Partido_barrio']]
print (data_antes.Partido_barrio.value_counts())
data_trabajo.loc[mask_geonames_id & mask_place_with_parent_names,['Partido_barrio']] = 'Almirante Brown'
print ('Cantidad de casos actualizados:')
data_despues = data_trabajo.loc[mask_geonames_id & mask_place_with_parent_names,['Partido_barrio']]
print (data_despues.Partido_barrio.value_counts())
# actualizamos el valor de Localidad para los casos de la mascara
print ('Cantidad de casos para actualizar:')
data_antes = data_trabajo.loc[mask_geonames_id & mask_place_with_parent_names,['Localidad']]
print (data_antes.Localidad.value_counts())
data_trabajo.loc[mask_geonames_id & mask_place_with_parent_names,['Localidad']] = 'San Jose'
print ('Cantidad de casos actualizados:')
data_despues = data_trabajo.loc[mask_geonames_id & mask_place_with_parent_names,['Localidad']]
print (data_despues.Localidad.value_counts())
# actualizamos el valor de place_name_new para los casos de la mascara
print ('Cantidad de casos para actualizar:')
data_antes =  data_trabajo.loc[mask_geonames_id & mask_place_with_parent_names,['place_name_new']]
print (data_antes.place_name_new.value_counts())
data_trabajo.loc[mask_geonames_id & mask_place_with_parent_names,['place_name_new']] = 'San Jose'
print ('Cantidad de casos actualizados:')
data_despues = data_trabajo.loc[mask_geonames_id & mask_place_with_parent_names,['place_name_new']]
print (data_despues.place_name_new.value_counts())



Comenzamos con el proceso de análisis y actualización de los los casos:
-----------------------------------------------------------------------
geonames_id = 3428927
----------------------
|Argentina|Bs.As. G.B.A. Zona Sur|Lomas de Zamora|San José|    11
|Argentina|Bs.As. G.B.A. Zona Sur|Almirante Brown|San Jose|     5
Name: place_with_parent_names, dtype: int64
Validamos que para esos place_with_parent_names no existan otros geonames_id
3428927.0    16
Name: geonames_id, dtype: int64
Resultado ... como son de la misma zona y no existen mask_place_with_parent_names para otros geonames_id
Validamos en google que la localidad de "San Jose" corresponde al partido de "Almirante Brown"
Para esos registros mask_place_with_parent_names = |Argentina|Bs.As. G.B.A. Zona Sur|Lomas de Zamora|San José|
Actualizamos la variable Partido_barrio = "Almirante Brown"
Actualizamos la variable Localidad = "San Jose"
Actualizamos la variable place_name_new = "San Jose"
Cantidad de casos para actualizar:
Lom

In [14]:
#..................................
# geonames_id = 3430234, 3433359, 3435548
    # mascara de id
print ('geonames_id = 3430234')
print ('----------------------')
mask_geonames_id  = data_trabajo.geonames_id == 3430234
# traemos la informacion de esos casos 
ver = data_trabajo.loc[mask_geonames_id,['place_with_parent_names']]
print (ver.place_with_parent_names.value_counts())
print ('Validamos que para esos place_with_parent_names no existan otros geonames_id')
    # Validamos que para esos place_with_parent_names no existan otros geonames_id
valores = ['|Argentina|Capital Federal|Palermo|','|Argentina|Capital Federal|Palermo|Palermo Soho|']
mask_place_with_parent_names  =  ([x in valores for x in data_trabajo.place_with_parent_names]) 
ver1 = data_trabajo.loc[mask_place_with_parent_names,['geonames_id']]
print (ver1.geonames_id.value_counts())
print ('Resultado ... como son de la misma zona y no existen mask_place_with_parent_names para otros geonames_id')
print ('Vamos a dejar geonames_id_new en nulo para el caso de "Palermo Soho" que es mas abarcativa para sumarlo') 
print ('Al tratamiente que vamos hacer despues con los nulos de geonames_id')
    # resultado ... como son de la misma zona y no existen mask_place_with_parent_names para otros geonames_id
    # vamos a dejar geonames_id_new en nulo para el caso de 'Palermo Soho' que es mas abarcativa para sumarlo 
    # al tratamiente que vamos hacer despues con los nulos de geonames_id
# mascara
mask_place_with_parent_names1 = data_trabajo.place_with_parent_names == '|Argentina|Capital Federal|Palermo|Palermo Soho|'
data_antes =   data_trabajo.loc[mask_geonames_id & mask_place_with_parent_names1, ['geonames_id_new']]
print ('Cantidad original de casos para actualizar:')
print(data_antes.geonames_id_new.value_counts())
data_trabajo.loc[mask_geonames_id & mask_place_with_parent_names1, ['geonames_id_new']] = None
print ('Cantidad posterior de casos para actualizar:')
data_despues =   data_trabajo.loc[mask_geonames_id & mask_place_with_parent_names1, ['geonames_id_new']]
print(data_despues.geonames_id_new.value_counts())

geonames_id = 3430234
----------------------
|Argentina|Capital Federal|Palermo|                 2885
|Argentina|Capital Federal|Palermo|Palermo Soho|     394
Name: place_with_parent_names, dtype: int64
Validamos que para esos place_with_parent_names no existan otros geonames_id
3430234.0    3279
Name: geonames_id, dtype: int64
Resultado ... como son de la misma zona y no existen mask_place_with_parent_names para otros geonames_id
Vamos a dejar geonames_id_new en nulo para el caso de "Palermo Soho" que es mas abarcativa para sumarlo
Al tratamiente que vamos hacer despues con los nulos de geonames_id
Cantidad original de casos para actualizar:
3430234.0    394
Name: geonames_id_new, dtype: int64
Cantidad posterior de casos para actualizar:
Series([], Name: geonames_id_new, dtype: int64)


In [15]:
#..................................
# geonames_id =  3433359
    # mascara de id
mask_geonames_id  = data_trabajo.geonames_id == 3433359
print ('geonames_id = 3433359')
print ('---------------------')
    # traemos la informacion de esos casos 
ver = data_trabajo.loc[mask_geonames_id,['place_with_parent_names'] ]
print (ver.place_with_parent_names.value_counts())
print ('Validamos que para esos place_with_parent_names no existan otros geonames_id')
    # Validamos que para esos place_with_parent_names no existan otros geonames_id
valores = ['|Argentina|Bs.As. G.B.A. Zona Oeste|Ituzaingó|','|Argentina|Bs.As. G.B.A. Zona Oeste|Ituzaingó|Ituzaingó|','|Argentina|Bs.As. G.B.A. Zona Oeste|Ituzaingó|Villa Udaondo|']
mask_place_with_parent_names  =  ([x in valores for x in data_trabajo.place_with_parent_names]) 
ver1 = data_trabajo.loc[mask_place_with_parent_names,['geonames_id']]
print (ver1.geonames_id.value_counts())
print ('Resultado ... como son de la misma zona y no existen mask_place_with_parent_names para otros geonames_id')
print ('Vamos a dejar geonames_id en nulo para el caso de "Villa Udaondo" que es mas abarcativa para sumarlo')
print ('Al tratamiente que vamos hacer despues con los nulos de geonames_id')
    # resultado ... como son de la misma zona y no existen mask_place_with_parent_names para otros geonames_id
    # vamos a dejar geonames_id en nulo para el caso de 'Villa Udaondo' que es mas abarcativa para sumarlo 
    # al tratamiente que vamos hacer despues con los nulos de geonames_id
# mascara
mask_place_with_parent_names1 = data_trabajo.place_with_parent_names == '|Argentina|Bs.As. G.B.A. Zona Oeste|Ituzaingó|Villa Udaondo|'
data_antes =   data_trabajo.loc[mask_geonames_id & mask_place_with_parent_names1, ['geonames_id_new']]
print ('Cantidad original de casos para actualizar:')
print(data_antes.geonames_id_new.value_counts())
data_trabajo.loc[mask_geonames_id & mask_place_with_parent_names1, ['geonames_id_new']] = None
print ('Cantidad posterior de casos para actualizar:')
data_despues =   data_trabajo.loc[mask_geonames_id & mask_place_with_parent_names1, ['geonames_id_new']]
print(data_despues.geonames_id_new.value_counts())


geonames_id = 3433359
---------------------
|Argentina|Bs.As. G.B.A. Zona Oeste|Ituzaingó|                  474
|Argentina|Bs.As. G.B.A. Zona Oeste|Ituzaingó|Ituzaingó|        358
|Argentina|Bs.As. G.B.A. Zona Oeste|Ituzaingó|Villa Udaondo|     25
Name: place_with_parent_names, dtype: int64
Validamos que para esos place_with_parent_names no existan otros geonames_id
3433359.0    857
Name: geonames_id, dtype: int64
Resultado ... como son de la misma zona y no existen mask_place_with_parent_names para otros geonames_id
Vamos a dejar geonames_id en nulo para el caso de "Villa Udaondo" que es mas abarcativa para sumarlo
Al tratamiente que vamos hacer despues con los nulos de geonames_id
Cantidad original de casos para actualizar:
3433359.0    25
Name: geonames_id_new, dtype: int64
Cantidad posterior de casos para actualizar:
Series([], Name: geonames_id_new, dtype: int64)


In [16]:
#..................................
# geonames_id = 3430234, 3433359, 3435548
    # mascara de id
print ('geonames_id = 3435548')
print ('---------------------')
mask_geonames_id  = data_trabajo.geonames_id == 3435548
    # traemos la informacion de esos casos 
ver = data_trabajo.loc[mask_geonames_id,['place_with_parent_names'] ]
print (ver.place_with_parent_names.value_counts())
print ('Validamos que para esos place_with_parent_names no existan otros geonames_id')
    # Validamos que para esos place_with_parent_names no existan otros geonames_id
valores = ['|Argentina|Buenos Aires Costa Atlántica|Mar del Plata|Centro|','|Argentina|Capital Federal|Centro / Microcentro|']
mask_place_with_parent_names  =  ([x in valores for x in data_trabajo.place_with_parent_names]) 
ver1 = data_trabajo.loc[mask_place_with_parent_names,['geonames_id'] ]
print (ver1.geonames_id.value_counts())
print ('Resultado ... en este caso no corresponde a la mimsa zona, pero no existen mask_place_with_parent_names para otros geonames_id')
print ('Vamos a dejar geonames_id en nulo para el caso de "Mar del Plata" solo para por simple elección y lo vamos a sumar')
print ('Al tratamiente que vamos hacer despues con los nulos de geonames_id')
    # resultado ... en este caso no corresponde a la mimsa zona, pero no existen mask_place_with_parent_names para otros geonames_id
    # vamos a dejar geonames_id en nulo para el caso de 'Mar del Plata' solo para por simple elección y lo vamos a sumar
    # al tratamiente que vamos hacer despues con los nulos de geonames_id
# mascara
mask_place_with_parent_names1 = data_trabajo.place_with_parent_names == '|Argentina|Buenos Aires Costa Atlántica|Mar del Plata|Centro|'
data_antes =   data_trabajo.loc[mask_geonames_id & mask_place_with_parent_names1, ['geonames_id_new']]
print ('Cantidad original de casos para actualizar:')
print(data_antes.geonames_id_new.value_counts())
data_trabajo.loc[mask_geonames_id & mask_place_with_parent_names1, ['geonames_id_new']] = None
print ('Cantidad posterior de casos para actualizar:')
data_despues =   data_trabajo.loc[mask_geonames_id & mask_place_with_parent_names1, ['geonames_id_new']]
print(data_despues.geonames_id_new.value_counts())



geonames_id = 3435548
---------------------
|Argentina|Buenos Aires Costa Atlántica|Mar del Plata|Centro|    231
|Argentina|Capital Federal|Centro / Microcentro|                 223
Name: place_with_parent_names, dtype: int64
Validamos que para esos place_with_parent_names no existan otros geonames_id
3435548.0    454
Name: geonames_id, dtype: int64
Resultado ... en este caso no corresponde a la mimsa zona, pero no existen mask_place_with_parent_names para otros geonames_id
Vamos a dejar geonames_id en nulo para el caso de "Mar del Plata" solo para por simple elección y lo vamos a sumar
Al tratamiente que vamos hacer despues con los nulos de geonames_id
Cantidad original de casos para actualizar:
3435548.0    231
Name: geonames_id_new, dtype: int64
Cantidad posterior de casos para actualizar:
Series([], Name: geonames_id_new, dtype: int64)


In [17]:
#..................................
# geonames_id = 3435907
    # mascara de id
print ('geonames_id = 3435907')
print ('---------------------')
valores_geonames_id = [3435907]
mask_geonames_id  = ([x in valores_geonames_id for x in data_trabajo.geonames_id]) 
    # traemos la informacion de esos casos 
ver = data_trabajo.loc[mask_geonames_id,['place_with_parent_names'] ]
print (ver.place_with_parent_names.value_counts())
print ('Validamos que para esos place_with_parent_names no existan otros geonames_id')
    # Validamos que para esos place_with_parent_names no existan otros geonames_id
valores = ['|Argentina|Bs.As. G.B.A. Zona Norte|','|Argentina|Bs.As. G.B.A. Zona Oeste|','|Argentina|Bs.As. G.B.A. Zona Sur|','|Argentina|Buenos Aires Costa Atlántica|','|Argentina|Buenos Aires Interior|']
mask_place_with_parent_names  =  ([x in valores for x in data_trabajo.place_with_parent_names]) 
ver1 = data_trabajo.loc[mask_place_with_parent_names,['geonames_id'] ]
print (ver1.geonames_id.value_counts())
print ('Resultado ... en este caso no corresponde a la mimsa zona, pero no existen mask_place_with_parent_names para otros geonames_id')
print ('Vamos a dejar geonames_id en nulo para todos los casos y lo vamos a sumar')
print ('Al tratamiente que vamos hacer despues con los nulos de geonames_id')
    # resultado ... en este caso no corresponde a la mimsa zona, pero no existen mask_place_with_parent_names para otros geonames_id
    # vamos a dejar geonames_id en nulo para todos los casos y lo vamos a sumar
    # al tratamiente que vamos hacer despues con los nulos de geonames_id
# mascara
data_antes =   data_trabajo.loc[mask_geonames_id, ['geonames_id_new']]
print ('Cantidad original de casos para actualizar:')
print(data_antes.geonames_id_new.value_counts())
data_trabajo.loc[mask_geonames_id, ['geonames_id_new']] = None
print ('Cantidad posterior de casos para actualizar:')
data_despues =   data_trabajo.loc[mask_geonames_id, ['geonames_id_new']]
print(data_despues.geonames_id_new.value_counts())

geonames_id = 3435907
---------------------
|Argentina|Bs.As. G.B.A. Zona Norte|        222
|Argentina|Buenos Aires Interior|           106
|Argentina|Bs.As. G.B.A. Zona Oeste|         65
|Argentina|Buenos Aires Costa Atlántica|     27
|Argentina|Bs.As. G.B.A. Zona Sur|           24
Name: place_with_parent_names, dtype: int64
Validamos que para esos place_with_parent_names no existan otros geonames_id
3435907.0    444
Name: geonames_id, dtype: int64
Resultado ... en este caso no corresponde a la mimsa zona, pero no existen mask_place_with_parent_names para otros geonames_id
Vamos a dejar geonames_id en nulo para todos los casos y lo vamos a sumar
Al tratamiente que vamos hacer despues con los nulos de geonames_id
Cantidad original de casos para actualizar:
3435907.0    444
Name: geonames_id_new, dtype: int64
Cantidad posterior de casos para actualizar:
Series([], Name: geonames_id_new, dtype: int64)


In [18]:
import random
print('Variable geonames_id')
print('Vamos a analizar como podemos actualizar los valores nulos')
print('----------------------------------------------------------')
print('Cantidad de valores nulos: ' + str(data_trabajo.geonames_id_new.isnull().sum()))
print('Cantidad de valores distintos: ' + str(len(data_trabajo.geonames_id_new.unique())))
print('Valor mínimo: ' + str(data_trabajo.geonames_id_new.min()))
print('Valor máximo: ' + str(data_trabajo.geonames_id_new.max()))
# vamos a trabajar los nulos del campo geonames_id_new\n# y lo hacemos por zonas:
# definimos 3 zonas de acuerdo al analisis del dato faltante de los procesos sobre place_with_parent_names
# valores_geo_grupo1  = \'Bs.As. G.B.A. Zona Norte\',\'Bs.As. G.B.A. Zona Oeste\',\'Bs.As. G.B.A. Zona Sur\',\'Capital Federal\'\n# mas valores_geo_interior\n    
    # si no tiene sino tiene Partido_barrio informado lo dejamos nulo ya que no seria correcto asignar un valor ya que \n    
    # abarca demasiado territorio\n    
    # si tiene Partido_barrio asigamos un numero al azar que identificara el place_name\n
# valores_geo_grupo2 = \'Buenos Aires Costa Atlántica\', \'Buenos Aires Interior\'\n    
    # si no tiene sino tiene Partido_barrio informado lo dejamos nulo ya que no seria correcto asignar un valor ya que \n    
    # abarca demasiado territorio\n    
    # si tiene Partido_barrio y Localidad, y NO son iguales asigamos un numero al azar que identificara el place_name\n    
    # si tiene Partido_barrio y Localidad, y SI son iguales asigamos, buscamos el ID del Partido_barrio \n    
# que si está informado\n\n
# creamos los grupos\n
valores_geo_grupo1 = ['Bs.As. G.B.A. Zona Norte','Bs.As. G.B.A. Zona Oeste','Bs.As. G.B.A. Zona Sur','Capital Federal']
valores_geo_grupo2 = ['Buenos Aires Costa Atlántica', 'Buenos Aires Interior']
valores_geo_interior_temp = data_trabajo.Zona.unique()
valores_geo_interior_y_1 = list(set(valores_geo_interior_temp).difference(set(valores_geo_grupo2)))

mask_geo_grupo2  =  ([x in valores_geo_grupo2 for x in data_trabajo.Zona])
mask_geo_interior_y_1  =  ([x in valores_geo_interior_y_1 for x in data_trabajo.Zona])
mask_geo_null = data_trabajo.geonames_id_new.isnull()
#mask_Partido_barrio_notnull = data_trabajo.Partido_barrio.notnull()
mask_Partido_barrio_null =  (data_trabajo.Partido_barrio.isnull()) | (data_trabajo.Partido_barrio == '')
mask_Partido_barrio_notnull = mask_data_Partido_barrio_null == False
mask_PB_dist_Localidad = data_trabajo.Partido_barrio != data_trabajo.Localidad

# Creamos una serie con los distintos valores de place_name que no interesa aplicarle una numero al azar
groupby_place_name_null = data_trabajo.loc[(mask_geo_interior_y_1 & mask_geo_null & mask_Partido_barrio_notnull)\
                                   | (mask_geo_grupo2&mask_geo_null&mask_Partido_barrio_notnull&mask_PB_dist_Localidad)\
                                   ].groupby(['place_name_new'])["Id_caso"].count()
# convertimos la seria en un df para trabajarlo
groupby_place_name_null_df = groupby_place_name_null.to_frame()
# asignamos a cada valor el número aleatorio partiendo de un numero lo bastante grande para que
# coincida con los existentes
groupby_place_name_null_df["geonames_id_temp"] = groupby_place_name_null_df.apply(lambda x: random.randint(10000000, 20000000), axis=1)

# creamos un nuevo df sumando el campo nuevo 
datay = pd.merge(left=data_trabajo,right=groupby_place_name_null_df, how='left', left_on='place_name_new', \
                 right_on='place_name_new')
#-------------------------------------------------------------------------------------------------------------
# actualizamos los casos que convertimos a nulo en el arreglo de geonames_id
#mask_geo_temp = ['|Argentina|Capital Federal|Palermo|Palermo Soho|']
#mask_geo_temp1  =  ([x in mask_geo_temp for x in datay.place_with_parent_names])
#max_valor = datay.geonames_id_temp.max()
#max_valor = max_valor + 1
#print(max_valor)
#datay.loc[mask_geo_temp1, 'geonames_id_temp'] = datay.loc[mask_geo_temp1, 'geonames_id_temp'].apply(lambda x: max_valor)
#['|Argentina|Buenos Aires Costa Atlántica|Mar del Plata|Centro|']
#['|Argentina|Bs.As. G.B.A. Zona Oeste|Ituzaingó|Villa Udaondo|']
#-------------------------------------------------------------------------------------------------------------

mask_geo_null = data_trabajo.geonames_id_new.isnull()
datay.loc[mask_geo_null, 'geonames_id_new'] = datay.loc[mask_geo_null, 'geonames_id_temp']
del datay['Id_caso_y']
datay.rename(columns={'Id_caso_x': 'Id_caso'}, inplace=True)
print('Cantidad de valores nulos final: ' + str(datay.geonames_id_new.isnull().sum()))
print('Cantidad de valores distintos final: ' + str(len(datay.geonames_id_new.unique())))

#Validamos que luego de la aplicacion de nulos no hayan quedado para un mismo 
# geonames_id a distintos valores de place_name_new

# variable geonames_id,  corresponde al id asignado a cada place_name
# volvemos a validar que No se asigne un mismo geonames_id a distintos valores de place_name_new
mask_geonames_id_new_notnull = datay.geonames_id_new.notnull()
df_geonames_id_place_name_ag2 = datay.loc[mask_geonames_id_new_notnull]

grouped_0 = df_geonames_id_place_name_ag2.groupby(['geonames_id_new',  'place_name_new'])
grouped_1 = grouped_0['Id_caso'].agg([np.size])

df_agrupado_2 = grouped_1.groupby(['geonames_id_new']).filter(lambda grp: grp["size"].count() > 1)
print ('Casos que un mismo geonames_id tiene asignado más de un place_name')
print ('------------------------------------------------------------------')
print (df_agrupado_2)

data_trabajo = datay
data_trabajo.drop("geonames_id_temp", axis = 1, inplace=True)

print ('Casos con geonames_id nulo')
print ('--------------------------')
# actualizo la mascara de los nulos
mask_geo_null = data_trabajo.geonames_id_new.isnull()
print (data_trabajo.loc[mask_geo_null, 'place_with_parent_names'].value_counts())




Variable geonames_id
Vamos a analizar como podemos actualizar los valores nulos
----------------------------------------------------------
Cantidad de valores nulos: 19811
Cantidad de valores distintos: 646
Valor mínimo: 3427208.0
Valor máximo: 6948895.0
Cantidad de valores nulos final: 610
Cantidad de valores distintos final: 1107
Casos que un mismo geonames_id tiene asignado más de un place_name
------------------------------------------------------------------
Empty DataFrame
Columns: [size]
Index: []
Casos con geonames_id nulo
--------------------------
|Argentina|Bs.As. G.B.A. Zona Norte|                                     222
|Argentina|Buenos Aires Interior|                                        106
|Argentina|Tucumán|                                                       77
|Argentina|Bs.As. G.B.A. Zona Oeste|                                      65
|Argentina|Buenos Aires Interior|Chascomús|Chascomús|                     54
|Argentina|Buenos Aires Costa Atlántica|           

In [129]:
print ('Proceso de validación de los campos Lat-Lon vs Lat / Lon')
print ('--------------------------------------------------------')
# validamos que los campos Lat y Lon tengan la misma cantidad y valor de info que Lat-Lon
# primero separamos el campo Lat-Lon
df_lat_lon  = data_trabajo.loc[:,["Id_caso","lat-lon", "lat", "lon"]].join(data["lat-lon"].str.split(',', expand=True).rename(columns={
                                                                                                0:'lat_temp', 
                                                                                                1:'lon_temp'}))
# los campos saparados los convierto en numéricos
df_lat_lon.lat_temp = df_lat_lon.lat_temp.apply(pd.to_numeric)
df_lat_lon.lon_temp = df_lat_lon.lon_temp.apply(pd.to_numeric)

# Validmos que los campos tengan la misma cantidad de NO nulos
print (df_lat_lon[["lat-lon", "lat", "lon"]].notnull().sum())
# mascara de no nulos
mask_notnull = df_lat_lon["lat-lon"].notnull()
# mascara de valores iguales 
ver_lat = round(df_lat_lon.lat,6) == round(df_lat_lon.lat_temp,6)
ver_lon = round(df_lat_lon.lon,6) == round(df_lat_lon.lon_temp,6)
# validamos la cantidad de casos
print ('Cantidad de casos iguales latitud: ' + str(ver_lat.sum()))
print ('Cantidad de casos iguales longitud: ' + str(ver_lon.sum()))


Proceso de validación de los campos Lat-Lon vs Lat / Lon
--------------------------------------------------------
lat-lon    69670
lat        69670
lon        69670
dtype: int64
Cantidad de casos iguales latitud: 69502
Cantidad de casos iguales longitud: 69396


In [21]:
print ('Proceso de validación de tipo de cambio')
print ('---------------------------------------')
# proceso de validación del tipo de cambio para validar si es correcto tomar el 
# el campo price_aprox_usd para todos los casos

# validamos que si currency (moneda) = 
# nulo  ----> nulo
# 'USD'  ----> price_aprox_local_currency / price  ----> valor por dolar de precios en dolares
# sino ---->price_aprox_local_currency / price_aprox_usd ----> valor por dolar de precios en pesos
# agrupamos los valores por dolar ... redondeado a 2 dígitos

#.................................................
print ('Distintos tipos de moneda:')
# Vemos los distintos valores de la variable moneda
print(data_trabajo.currency.value_counts())
# vemos que existen 3 registros con valores PEN (sol peruano) y UYU (peso Uruguayo)
# vemos los casos
print ('Los valores "PEN" y "UYU" los actualizamos a nulos para no descartarlos')
valores_otras_monedas = ['PEN', 'UYU']
mask_otras_monedas = [x in valores_otras_monedas for x in data_trabajo.currency]
data_trabajo.loc[mask_otras_monedas, ["currency","price"]] = None

# sumamos la varible al Df Valor_dolar
data_trabajo["Valor_dolar"] = 0
# trabajamos la moneda = 'USD'
mask_dolar = data_trabajo.currency == 'USD'
mask_price = (data_trabajo.price != 0) | (data_trabajo.price.notnull())
data_trabajo.loc[mask_dolar & mask_price, "Valor_dolar"] = round(data_trabajo.price_aprox_local_currency / data_trabajo.price,2)
# trabajamos la moneda = 'ARG'
mask_peso = data_trabajo.currency == 'ARS'
mask_price_aprox_usd = (data_trabajo.price_aprox_usd != 0) | (data_trabajo.price_aprox_usd.notnull())
data_trabajo.loc[mask_peso & mask_price_aprox_usd, "Valor_dolar"] = round(data_trabajo.price_aprox_local_currency / data_trabajo.price_aprox_usd,2)
print ('Distintos valores del tipo de cambio luego de actualizar la variable creada')
# vemos los valores distintos para valor de dolar
print(data_trabajo.Valor_dolar.value_counts())
print ('Vemos que existe un solo valor ... Exc!')
print ('Llegamos a la conclusion que es posible usar la variable price_aprox_usd como precio de la propiedad estándar')




Proceso de validación de tipo de cambio
---------------------------------------
Distintos tipos de moneda:
USD    87587
ARS    13219
PEN        2
UYU        1
Name: currency, dtype: int64
Los valores "PEN" y "UYU" los actualizamos a nulos para no descartarlos
Distintos valores del tipo de cambio luego de actualizar la variable creada
17.64    100806
0.00      20414
Name: Valor_dolar, dtype: int64
Vemos que existe un solo valor ... Exc!
Llegamos a la conclusion que es posible usar la variable price_aprox_usd como precio de la propiedad estándar


In [22]:
print ('Proceso de análisis del precio por m2')
print ('-------------------------------------')
print('Variables:')
print('- price_usd_per_m2')
print('- price_per_m2')
    
print ('Al ver alguno ejemplos vamos a validar que:')
print ('price_usd_per_m2 corresponde al precio en dólares sobre la superficie total en m2(price_aprox_usd / surface_total_in_m2)')
print ('y que price_per_m2 corresponde al precio original sobre la superficie cubierta(price / surface_covered_in_m2)')
print ('por lo que sobre esta última, ademas vamos a estandarizarla a dólares sobre una nueva variable')
    

# lo validamos
print ('Validación de price_usd_per_m2')
print ('------------------------------')
# ----------------------------------price_usd_per_m2
# sumamos un nuevo campo 
data_trabajo["price_usd_per_m2_new"] = -1
# mascara de los NO nulos price_aprox_usd
mask_price_aprox_usd_notnull = data_trabajo.price_aprox_usd.notnull()
# mascara de los no nulos y mayores a cero de surface_total_in_m2
mask_surface_total_in_m2_notnull_0 = ((data_trabajo.surface_total_in_m2.notnull()) & (data_trabajo.surface_total_in_m2 > 0))
# actualización de la variable nueva price_usd_per_m2_new
data_trabajo.loc[mask_price_aprox_usd_notnull & mask_surface_total_in_m2_notnull_0, "price_usd_per_m2_new"] = round(data_trabajo.price_aprox_usd/data_trabajo.surface_total_in_m2,2)
# validamos creando una mascara sobre los que NO coinciden variable original vs variable nueva
mask_validacion_price_aprox_usd = data_trabajo.price_usd_per_m2_new != round(data_trabajo.price_usd_per_m2,2)
print('Luego de "imitar" la variable agrupamos por la marca si coinciden o no con la variable original:')
# agrupamos por la mascara y vemos que existen 52606 casos que son verdaderos .. NO coinciden
print(mask_validacion_price_aprox_usd.value_counts())
print('Vemos que hay 52606 que no coinciden')
print('Validamos que todos los casos que correspondan a los valores nulos, mediante una máscara')
# contamos los nulos de esos no coincidentes y vemos que no coinciden porque la variable nueva se 
# le asigne como valor default -1 y en la variable original estan con nulo
print(data_trabajo.loc[mask_validacion_price_aprox_usd, "price_usd_per_m2"].isnull().value_counts())
print('Conclusión: la variable price_usd_per_m2 corresponde a')
print('a price_aprox_usd / surface_total_in_m2')

# ----------------------------------price_per_m2
print ('Validación de price_per_m2')
print ('------------------------------')
# sumamos un nuevo campo 
data_trabajo["price_per_m2_new"] = -1
# mascara de los NO nulos price
mask_price_notnull = data_trabajo.price.notnull()
# mascara de los no nulos y mayores a cero de surface_covered_in_m2
mask_surface_covered_in_m2_notnull_0 = ((data_trabajo.surface_covered_in_m2.notnull()) & (data_trabajo.surface_covered_in_m2 > 0))
# actualización de la variable nueva price_per_m2_new
data_trabajo.loc[mask_price_notnull & mask_surface_covered_in_m2_notnull_0, "price_per_m2_new"] = round(data_trabajo.price/data_trabajo.surface_covered_in_m2,2)
# validamos creando una mascara sobre los que NO coinciden variable original vs variable nueva
mask_validacion_price_per_m2 = data_trabajo.price_per_m2_new != round(data_trabajo.price_per_m2,2)
print('Luego de "imitar" la variable agrupamos por la marca si coinciden o no con la variable original:')
# agrupamos por la mascara y vemos que existen 33562 casos que son verdaderos .. NO coinciden
print(mask_validacion_price_per_m2.value_counts())
print('Vemos que hay 33562 que no coinciden')
print('Validamos que todos los casos que correspondan a los valores nulos, mediante una máscara')
# contamos los nulos de esos no coincidentes y vemos que no coinciden porque la variable nueva se 
# le asigne como valor default -1 y en la variable original estan con nulo
print (data_trabajo.loc[mask_validacion_price_per_m2, "price_per_m2"].isnull().value_counts())
print('Conclusión: la variable price_per_m2 corresponde a')
print('price / surface_covered_in_m2')
# ahora vamos a actualizar la veriable que creamos price_per_m2 pero tomando para el caso de moneda ARG la 
# variable en dolares price_aprox_usd para que todo quede en la misma moneda
# mascara moneda ARG
mask_peso = data_trabajo.currency == 'ARS'
data_trabajo.loc[mask_peso & mask_surface_covered_in_m2_notnull_0, "price_per_m2_new"] = round(data_trabajo.price_aprox_usd/data_trabajo.surface_covered_in_m2,2)
# pasamos a nulo los que quedaron con valor -1
data_trabajo['price_usd_per_m2_new'].replace(-1, np.nan, inplace=True)
data_trabajo['price_per_m2_new'].replace(-1, np.nan, inplace=True)


Proceso de análisis del precio por m2
-------------------------------------
Variables:
- price_usd_per_m2
- price_per_m2
Al ver alguno ejemplos vamos a validar que:
price_usd_per_m2 corresponde al precio en dólares sobre la superficie total en m2(price_aprox_usd / surface_total_in_m2)
y que price_per_m2 corresponde al precio original sobre la superficie cubierta(price / surface_covered_in_m2)
por lo que sobre esta última, ademas vamos a estandarizarla a dólares sobre una nueva variable
Validación de price_usd_per_m2
------------------------------
Luego de "imitar" la variable agrupamos por la marca si coinciden o no con la variable original:
False    68614
True     52606
dtype: int64
Vemos que hay 52606 que no coinciden
Validamos que todos los casos que correspondan a los valores nulos, mediante una máscara
True     52603
False        3
Name: price_usd_per_m2, dtype: int64
Conclusión: la variable price_usd_per_m2 corresponde a
a price_aprox_usd / surface_total_in_m2
Validación de price

In [137]:
########### esto no se ejecuta!!!
## de aca en adelante son pruebas


BUSCAR DUPLICADOS EN UN DF
artist_duplicated = data_artist.duplicated(subset=["Artist Display Name", "Artist Role"])
any(artist_duplicated) * me devuelve verdadero o falso
artist_duplicated.sum()  * me devuelve la cantidad de duplicados
BORRO LOS DUPLICADOS
data_artist_unique = data_artist.drop_duplicates(subset=["Artist Display Name", "Artist Role"], keep="first")
keep ... 
* first: devuelve False (no duplicado) el primer registro encontrado, y True (duplicados) todas las demás apariciones
* last: devuelve False (no duplicado) el último registro encontrado, y True (duplicados) todas las demás apariciones
* False: devuelve True (duplicado) todos los registros duplicados

In [152]:
ver = data_trabajo.duplicated(subset=["currency"])
data_artist_unique = data_artist.drop_duplicates(subset=["Artist Display Name", "Artist Role"], keep="first")

0         False
1          True
2          True
3          True
4          True
          ...  
121215     True
121216     True
121217     True
121218     True
121219     True
Length: 121220, dtype: bool

In [ ]:
df_na.to_csv('data_output/surveys_complete.csv', index=False)